In [1]:
# IMPORT

import pandas as pd
import pylab as plt
import seaborn as sns
import numpy as np
import h2o

from sklearn.model_selection import train_test_split
from h2o.automl import H2OAutoML
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2

import warnings
warnings.filterwarnings('ignore')

In [2]:
# LINEAR REGRESSION MODELS

from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
lasso=Lasso()
ridge=Ridge()
elastic=ElasticNet()
le=LabelEncoder

from sklearn.svm import SVR
svr=SVR()

from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
rfr=RFR()
etr=ETR()

from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor as XGBR
from catboost import CatBoostRegressor as CTR
from lightgbm import LGBMRegressor as LGBMR
gbr=GBR()
xgbr=XGBR()
ctr=CTR()
lgbmr=LGBMR()

In [3]:
# LOAD CSVs

train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
sample = pd.read_csv('../data/sample.csv')

In [4]:
train1 = train.copy()
test1 = test.copy()
sample1 = sample.copy()

In [5]:
train.corr().price

id                                             -0.000289
scrape_id                                            NaN
host_id                                         0.020774
host_listings_count                            -0.005046
host_total_listings_count                      -0.005046
neighbourhood_group_cleansed                         NaN
latitude                                        0.005321
longitude                                       0.022274
accommodates                                    0.330000
bathrooms                                            NaN
bedrooms                                        0.318763
beds                                            0.341547
price                                           1.000000
minimum_nights                                 -0.005230
maximum_nights                                  0.003689
minimum_minimum_nights                         -0.005431
maximum_minimum_nights                         -0.007911
minimum_maximum_nights         

In [ ]:
# BUSCANDO COINEALIDAD

plt.figure(figsize=(200, 200))

sns.set(style='white')

mask=np.triu(np.ones_like(train1.corr(), dtype=bool))

cmap=sns.diverging_palette(0, 10, as_cmap=True)

sns.heatmap(train1.corr(),
            mask=mask,
            cmap=cmap,
            center=0,
            square=True,
            linewidths=0.5,
            cbar_kws={'shrink': 0.5},
            annot=True);

In [24]:
'''le = LabelEncoder()

for c in train1.columns:
    
    if c=='host_location' or c=='host_verifications':
        
        le.fit(train1[c].astype(str))
        
        train1[c]=le.transform(train1[c].astype(str))'''

"le = LabelEncoder()\n\nfor c in train1.columns:\n    \n    if c=='host_location' or c=='host_verifications':\n        \n        le.fit(train1[c].astype(str))\n        \n        train1[c]=le.transform(train1[c].astype(str))"

In [25]:
train1=pd.get_dummies(train1, columns=['property_type'], drop_first=True)
lista = [col for col in train1.columns if 'property_type' in col]
train1[lista] = train1[lista].astype(float)

train1=pd.get_dummies(train1, columns=['bathrooms_text'], drop_first=True)
lista = [col for col in train1.columns if 'bathrooms_text' in col]
train1[lista] = train1[lista].astype(float)

train1=pd.get_dummies(train1, columns=['neighbourhood_cleansed'], drop_first=True)
lista = [col for col in train1.columns if 'neighbourhood_cleansed' in col]
train1[lista] = train1[lista].astype(float)

train1=pd.get_dummies(train1, columns=['host_neighbourhood'], drop_first=True)
lista = [col for col in train1.columns if 'host_neighbourhood' in col]
train1[lista] = train1[lista].astype(float)

In [26]:
def trans(train1):    
    
    if 'price' in train1.columns:
    
        # CHANGE PRICE COLUMN POSITION

        aux = train1.price

        train1 = train1.drop('price',axis=1)

        train1.insert(1, 'price', aux)

        train1 = train1[train1.price<=6000]

    # TRANSFORM COLUMNS

    train1.host_acceptance_rate = train1.host_acceptance_rate.str.replace('%','').astype(float)/100

    train1.host_response_rate = train1.host_response_rate.str.replace('%','').astype(float)/100

    # DROP TRAIN COLUMNS

    col_importance = ['property_type','neighbourhood_cleansed','host_verifications','host_neighbourhood',
                      'availability_365','longitude','bedrooms','accommodates','host_acceptance_rate','host_location',
                      'minimum_maximum_nights','availability_90','review_scores_value','host_total_listings_count',
                      'availability_30','number_of_reviews','host_id','first_review','bathrooms_text','host_response_rate']

    col_classification = [c for c in train1.columns if np.dtype(train1[c]) is np.dtype(np.object)]

    col_noutil = ['listing_url','scrape_id','last_scraped','name','description','neighborhood_overview','picture_url',
                  'host_url']

    train1 = train1.dropna(axis=1, how='all')
    train1 = train1.drop(col_noutil, axis=1)
    train1 = train1.drop(columns=[c for c in train1 if c in col_classification and c!='price' and c!='host_acceptance_rate'
                                  and c!='host_response_rate'], axis=1)
    # No correlación con price
    train1 = train1.drop(columns='id', axis=1)
    # >0.9 Correlación con otras columnas
    train1 = train1.drop('minimum_nights', axis=1)
    train1 = train1.drop('minimum_minimum_nights', axis=1)
    train1 = train1.drop('minimum_nights_avg_ntm', axis=1)
    train1 = train1.drop('availability_90', axis=1)
    train1 = train1.drop('availability_60', axis=1)
    train1 = train1.drop('host_total_listings_count', axis=1)
    train1 = train1.drop('host_listings_count', axis=1)

    # FILLNA columnas numéricas con sus medias

    col_somenull = [c for c in train1.columns if (train1[c].isnull().sum()>0)]

    for c in col_somenull:

        # Probar a cambiar mean por mode o median
        train1[c].fillna(round(train1[c].median()), inplace=True)
        
    return train1

train1 = trans(train1)

In [ ]:
'''le = LabelEncoder()

for c in test1.columns:
    
    if c=='host_location' or c=='host_verifications':
        
        le.fit(test1[c].astype(str))
        
        test1[c]=le.transform(test1[c].astype(str))'''

In [27]:
test1=pd.get_dummies(test1, columns=['property_type'], drop_first=True)
lista = [col for col in test1.columns if 'property_type' in col]
test1[lista] = test1[lista].astype(float)

test1=pd.get_dummies(test1, columns=['host_neighbourhood'], drop_first=True)
lista = [col for col in test1.columns if 'host_neighbourhood' in col]
test1[lista] = test1[lista].astype(float)

test1=pd.get_dummies(test1, columns=['bathrooms_text'], drop_first=True)
lista = [col for col in test1.columns if 'bathrooms_text' in col]
test1[lista] = test1[lista].astype(float)

test1=pd.get_dummies(test1, columns=['neighbourhood_cleansed'], drop_first=True)
lista = [col for col in test1.columns if 'neighbourhood_cleansed' in col]
test1[lista] = test1[lista].astype(float)

In [ ]:
test1 = trans(test1)

In [28]:
X = train1.drop('price', axis=1)

y = train1.price

X.shape, y.shape

((4165, 174), (4165,))

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, train_size=0.8,
                                                    random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3332, 174), (833, 174), (3332,), (833,))

In [30]:
# RMSE POR MODELO
# modelo que da error: lgbmr
models = [lasso, ridge, elastic, svr, rfr, etr, gbr, xgbr, ctr]

for m in models:
    
    m.fit(X_train, y_train)
    y_pred = m.predict(X_test)
    
    print(f'RMSE Model {m}: {mse(y_test, y_pred, squared=False)}')
    print(f'R2 Model {m}: {r2(y_test, y_pred)}')

RMSE Model Lasso(): 80.11320207996663
R2 Model Lasso(): 0.5139703914318152
RMSE Model Ridge(): 77.28230108844434
R2 Model Ridge(): 0.5477124478005073
RMSE Model ElasticNet(): 84.2147491349618
R2 Model ElasticNet(): 0.46293003661637044
RMSE Model SVR(): 118.86981399576307
R2 Model SVR(): -0.07003433158161476
RMSE Model RandomForestRegressor(): 75.77510874951305
R2 Model RandomForestRegressor(): 0.565181831478528
RMSE Model ExtraTreeRegressor(): 99.0344865782255
R2 Model ExtraTreeRegressor(): 0.25727600526719296
RMSE Model GradientBoostingRegressor(): 77.92981118246605
R2 Model GradientBoostingRegressor(): 0.5401017110894908
RMSE Model XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
           

In [31]:
'''models = [lasso, ridge, elastic, svr, rfr, etr, gbr, xgbr, lgbmr, ctr]

for m in models:
    
    m.fit(X, y)
    
    y_pred = m.predict(X)
    
    print(f'RMSE Model {m}: {mse(y, y_pred, squared=False)}')'''

"models = [lasso, ridge, elastic, svr, rfr, etr, gbr, xgbr, lgbmr, ctr]\n\nfor m in models:\n    \n    m.fit(X, y)\n    \n    y_pred = m.predict(X)\n    \n    print(f'RMSE Model {m}: {mse(y, y_pred, squared=False)}')"

In [35]:
# H2O

h2o.init()

# parsear datos para h20

h2train=h2o.H2OFrame(train1)

h2test=h2o.H2OFrame(test1)

X=[c for c in h2train.columns if c!='price']

y='price'

# inicia auto-machine-learning

automl=H2OAutoML(max_models=100,
                 seed=42,
                 max_runtime_secs=1800,
                 sort_metric='RMSE')

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 hour 35 mins
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,26 days
H2O_cluster_name:,H2O_from_python_Gaspar_n9zrcn
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.550 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [36]:
# entrena

automl.train(x=X,
             y=y,
             training_frame=h2train)

AutoML progress: |█
20:58:32.404: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_1_AutoML_4_20220509_205832_model_2


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,43.0,43.0,46690.0,11.0,11.0,11.0,32.0,141.0,81.34884




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 3396.399159959334
RMSE: 58.27863382028901
MAE: 32.20812569626239
RMSLE: 0.2937340872456456
Mean Residual Deviance: 3396.399159959334

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 6892.0817328422845
RMSE: 83.01856257995729
MAE: 46.7755993969312
RMSLE: 0.3937573062156734
Mean Residual Deviance: 6892.0817328422845

Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,47.006670,0.785142,47.368150,46.198673,46.487790,48.177494,46.801243
1,mean_residual_deviance,6960.015000,519.159550,6629.334000,6590.938500,7034.474000,6710.750500,7834.579000
2,mse,6960.015000,519.159550,6629.334000,6590.938500,7034.474000,6710.750500,7834.579000
3,r2,0.484658,0.037470,0.514163,0.527227,0.443775,0.488756,0.449371
4,residual_deviance,6960.015000,519.159550,6629.334000,6590.938500,7034.474000,6710.750500,7834.579000
5,rmse,83.381880,3.057111,81.420720,81.184590,83.871770,81.919174,88.513160
6,rmsle,0.393660,0.034467,0.357296,0.394004,0.366517,0.406561,0.443922



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
0,,2022-05-09 21:01:50,23.488 sec,0.0,116.305456,75.494801,13526.959014
1,,2022-05-09 21:01:50,23.867 sec,5.0,93.601569,57.318020,8761.253630
2,,2022-05-09 21:01:50,24.203 sec,10.0,82.098029,47.846722,6740.086415
3,,2022-05-09 21:01:51,24.522 sec,15.0,74.942503,42.102678,5616.378820
4,,2022-05-09 21:01:51,24.839 sec,20.0,70.111487,38.553056,4915.620555
5,,2022-05-09 21:01:51,25.136 sec,25.0,65.923596,36.060310,4345.920560
6,,2022-05-09 21:01:52,25.429 sec,30.0,63.047591,34.517448,3974.998672
7,,2022-05-09 21:01:52,25.712 sec,35.0,60.991389,33.324039,3719.949555
8,,2022-05-09 21:01:52,25.981 sec,40.0,59.430674,32.792548,3532.005030
9,,2022-05-09 21:01:52,26.159 sec,43.0,58.278634,32.208126,3396.399160



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,accommodates,48332028.00,1.000000,0.240694
1,bedrooms,36692660.00,0.759179,0.182730
2,availability_30,15207956.00,0.314656,0.075736
3,review_scores_location,7417485.00,0.153469,0.036939
4,longitude,7173694.00,0.148425,0.035725
5,availability_365,6563303.00,0.135796,0.032685
6,host_id,6189193.00,0.128056,0.030822
7,reviews_per_month,5396899.50,0.111663,0.026877
8,latitude,5342722.00,0.110542,0.026607
9,review_scores_value,5122120.50,0.105978,0.025508



See the whole table with table.as_data_frame()


In [37]:
# prediciones del lider

y_pred = automl.leader.predict(h2test)

y_pred = y_pred.as_data_frame()

sample1.price = y_pred

sample1.to_csv('../data/predictions.csv', index=False)

sample1

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


,price,id
0,130.126753,0
1,82.255555,1
2,80.039620,2
3,115.540054,3
4,128.011120,4
...,...,...
1384,131.802244,1384
1385,59.463762,1385
1386,133.519578,1386
1387,216.855689,1387


In [6]:
# buscando colinealidad

plt.figure(figsize=(200, 200))

sns.set(style='white')

mask=np.triu(np.ones_like(train1.corr(), dtype=bool))

cmap=sns.diverging_palette(0, 10, as_cmap=True)

sns.heatmap(train1.corr(),
            mask=mask,
            cmap=cmap,
            center=0,
            square=True,
            linewidths=0.5,
            cbar_kws={'shrink': 0.5},
            annot=True);